In [23]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import re
import warnings
warnings.filterwarnings('ignore')
import json
import requests
from urllib.request import urlopen
import math

In [70]:
#Initiating webdriver
def initiate_driver():
    driver=webdriver.Chrome('C:/Users/yn/Desktop/Yuvi/DataTrained/seleniumWebdriver/chromedriver.exe')
    return driver

In [71]:
#Function to load the url and parse the json response using webdriver
def car_info_json_using_webdriver(driver,url):   
    try:
        driver.get(url)
        json_content=driver.find_element_by_xpath('//body').text
        json_parser=json.loads(json_content)
        return json_parser
    except:
        raise("Expection Occured while loading the web page")
        

In [49]:
#Funstion to load the url and parse the json response using urlopen
def car_info_json(url):
    json_content=urlopen(url)
    print(json_content)
    json_parser=json.loads(json_content.read())
    return json_parser

In [ ]:
#Collect car info from CarDheko website
def get_cardheko_info(url):
    try:
        json_data=car_info_json(url)
    except:
        raise("Exception occured while collecting the json data")
    #Empty list for collecting brand info
    brand_list=[]
    #Empty list for collecting model info
    model_list=[]
    #Empty list for collecting variant info
    variant_list=[]
    #Empty list for collecting body type info
    body_type_list=[]
    #Empty list for collecting location info
    location_list=[]
    #Empty list for collecting km driven info
    km_driven_list=[]
    #Empty list for collecting no.of.owners info
    no_of_owners_list=[]
    #Empty list for collecting transmission type info
    transmission_type_list=[]
    #Empty list for collecting fuel type info
    fuel_type_list=[]
    #Empty list for collecting manufacturing year info
    manf_year_list=[]
    #Empty list for collecting price info
    price_list=[]
    #print(len(json_data['data']['cars']))
    for car in json_data['data']['cars']:
        if 'first-owner' in url:
            no_of_owners_list.append('1st Owner')
        elif 'second-owner' in url:
            no_of_owners_list.append('2nd Owner')
        try:
            brand_list.append(car['oem'])
        except:
            brand_list.append('-')
        try:
            model_list.append(car['model'])
        except:
            model_list.append('-')
        try:
            variant_list.append(car['carVariant'])
        except:
            variant_list.append('-')
        try:
            body_type_list.append(car['bt'])
        except:
            body_type_list.append('-')
        try:
            location_list.append(car['city'])
        except:
            location_list.append('-')
        try:
            km_driven_list.append(car['km'])
        except:
            km_driven_list.append('-')
        try:
            transmission_type_list.append(car['transmissionType'])
        except:
            transmission_type_list.append('-')
        try:
            fuel_type_list.append(car['ft'])
        except:
            fuel_type_list.append('-')
        try:
            manf_year_list.append(car['myear'])
        except:
            manf_year_list.append('-')
        try:
            price_list.append(car['used_car_price'])
        except:
            price_list.append('-')
    car_table=pd.DataFrame({"Brand":brand_list,"Model":model_list,"Variant":variant_list,"Body Type":body_type_list,
                            "Location":location_list,"Km Driven":km_driven_list,"Transmission Type":transmission_type_list,
                            "Fuel Type":fuel_type_list,"Manufacture Year":manf_year_list,"No.Of.Owners":no_of_owners_list,"Price":price_list
                           })
    return car_table

In [4]:
#Fucntion to scrap Car Dheko Cars info
def car_dheko_cars():
    #Api url for collecting json response for all the first owner type cars
    first_owner_url_json='https://listing.cardekho.com/v1/srp/cardekho?&lang_code=en&regionId=0&searchstring=used-cars%2Bfirst-owner&pagefrom=0'
    #Api url for collecting json response for all the second owner type cars
    second_owner_url_json='https://listing.cardekho.com/v1/srp/cardekho?&lang_code=en&regionId=0&searchstring=used-cars%2Bsecond-owner&pagefrom=0'
    
    #Api urls list for first and second owners cars
    start_urls=[first_owner_url_json,second_owner_url_json]
    #Empty dataframe for final result
    cars_df=pd.DataFrame()
    
    #looping through the urls list
    for j_url in start_urls:
        if 'pagefrom=0' in j_url:
            #Collecting total number of items count
            list_count=car_info_json(j_url)['data']['count']
            print(list_count)
            #Finding the total number of pages by dividing the total count with per page count
            total_pages=math.ceil(int(list_count)/20)+1
            print(total_pages)
            i=0
            #looping through all total number of pages
            while(total_pages!=0):
                #Reframing the url for every page by changing the page number
                cur_page_url=j_url[:j_url.rindex('pagefrom=')]+'pagefrom='+str(i)                
                try:
                    #Collecting the parsed data
                    df1=get_cardheko_info(cur_page_url)
                    #Keep appending the result to the final data frame
                    cars_df=cars_df.append(df1,ignore_index=True)
                except:
                    print("exception occured while collecting data from Cars Dheko")
                total_pages=total_pages - 1
                i+=1
        print(cars_df.shape)
    cars_df.to_csv('C:/Users/yn/Desktop/Yuvi/DataTrained/InternshipProjects/CarPricePrediction/CarPriceDataSet.csv')
    return cars_df

In [ ]:
car_dheko_cars()

In [79]:
# collect car info from Carwala
def get_car_wala_info(driver,url,body_type_dict,trans_type_dict):    
    #Get car info from json response
    try:
        json_data=car_info_json_using_webdriver(driver,url)
    except:
        raise("Exception occured while collecting the json data")
    #Empty list for collecting brand info
    brand_list=[]
    #Empty list for collecting model info
    model_list=[]
    #Empty list for collecting variant info
    variant_list=[]
    #Empty list for collecting body type info
    body_type_list=[]
    #Empty list for collecting location info
    location_list=[]
    #Empty list for collecting km driven info
    km_driven_list=[]
    #Empty list for collecting no.of.owners info
    no_of_owners_list=[]
    #Empty list for collecting transmission type info
    transmission_type_list=[]
    #Empty list for collecting fuel type info
    fuel_type_list=[]
    #Empty list for collecting manufacturing year info
    manf_year_list=[]
    #Empty list for collecting price info
    price_list=[]
    #print(len(json_data['data']['cars']))
    for car in json_data['ResultData']:
        if 'trans=1' in url:
            transmission_type_list.append(trans_type_dict['1'])
        elif 'trans=2' in url:
            transmission_type_list.append(trans_type_dict['2'])
        else:
            transmission_type_list.append('-')
        try:
            brand_list.append(car['MakeName'])
        except:
            brand_list.append('-')
        try:
            model_list.append(car['ModelName'])
        except:
            model_list.append('-')
        try:
            variant_list.append(car['VersionName'])
        except:
            variant_list.append('-')
        try:
            body_type_id=car['BodyStyleId']
            body_type_list.append(body_type_dict[body_type_id])
        except:
            body_type_list.append('-')
        try:
            no_of_owners_list.append(car['NoOfOwners'])
        except:
            no_of_owners_list.append('-')
        try:
            location_list.append(car['CityName'])
        except:
            location_list.append('-')
        try:
            km_driven_list.append(car['KmNumeric'])
        except:
            km_driven_list.append('-')
        try:
            fuel_type_list.append(car['Fuel'])
        except:
            fuel_type_list.append('-')
        try:
            manf_year_list.append(car['MakeYear'])
        except:
            manf_year_list.append('-')
        try:
            price_list.append(car['PriceNumeric'])
        except:
            price_list.append('-')
    car_table=pd.DataFrame({"Brand":brand_list,"Model":model_list,"Variant":variant_list,"Body Type":body_type_list,
                            "Location":location_list,"Km Driven":km_driven_list,"Transmission Type":transmission_type_list,
                            "Fuel Type":fuel_type_list,"Manufacture Year":manf_year_list,"No.Of.Owners":no_of_owners_list,"Price":price_list
                           })
    return car_table

In [80]:
#Function to scrap car infor from car wala
def car_wala_cars():
    #Api url to get the json response 
    j_url='https://www.carwale.com/webapi/classified/stockfilters/?lcr=0'
    #Initiating selenium webdriver
    driver=initiate_driver()
    #loading the html web page to collect the body type list from website
    driver.get('https://www.carwale.com/used/cars-for-sale/#sc=-1&so=-1&pn=1')
    time.sleep(10)
    #Empty dictionary to collect body type id and corresponding name
    body_type_dict={}  
    #Get BodyType from html url
    body_type_list=driver.find_elements_by_xpath('//div[@name="bodytype"]/div/ul[1]/li') 
    #looping through list of body types available in web page
    for body_type in body_type_list:
        body_type_dict[body_type.get_attribute('filterid')]=body_type.get_attribute('name')
    
    #Get TransType from html url
    trans_type_list=driver.find_elements_by_xpath('//div[@name="trans"]/div/ul[1]/li')
    print(len(trans_type_list))
    #Empty dict to collect transmission type id and it's corresponding name
    trans_type_dict={}
    #looping through list of transmission type available in web page
    for trans_type in trans_type_list:
        trans_type_dict[trans_type.get_attribute('filterid')]=trans_type.get_attribute('name')
    #Empty data frame for final result
    cars_df=pd.DataFrame()
    print(len(trans_type_dict))
    
    #looping through the different transmission types available
    for key,value in trans_type_dict.items():
        #reframing the api url based on the transmission type id
        j_url='https://www.carwale.com/webapi/classified/stockfilters/?lcr=0&trans='+key
        print(j_url)
        if 'lcr=0' in j_url:
            #collecting the total number of items count
            count=int(car_info_json_using_webdriver(driver,j_url)['FiltersData']['stockcount']['TotalStockCount'])
        print(count)
        i=0
        #looping through until reaches the max count
        while(i<count):
            #increamenting the offset value with 24 every iteration
            i=int(i)+24
            j_url=j_url.split('?lcr=')[0]+'?lcr='+str(i)+j_url[j_url.rindex('&trans='):]
            try:
                #Keep appending the result to the final dataframe
                cars_df=cars_df.append(get_car_wala_info(driver,j_url,body_type_dict,trans_type_dict),ignore_index=True)
            except:
                print("Exception occured while collecting info from Car wala")
    print(cars_df.shape)
    cars_df.to_csv('C:/Users/yn/Desktop/Yuvi/DataTrained/InternshipProjects/CarPricePrediction/CarPriceDataSetCarWala.csv')
    return cars_df

In [81]:
car_wala_cars()

2
2
https://www.carwale.com/webapi/classified/stockfilters/?lcr=0&trans=2
76961
https://www.carwale.com/webapi/classified/stockfilters/?lcr=0&trans=1
14467
(50389, 11)


,Brand,Model,Variant,Body Type,Location,Km Driven,Transmission Type,Fuel Type,Manufacture Year,No.Of.Owners,Price
0,Maruti Suzuki,Ciaz [2017-2018],Zeta 1.4 MT,Sedan,Bhopal,44000,Manual,Petrol,2017,First Owner,665000
1,Hyundai,Grand i10 [2013-2017],Asta 1.2 Kappa VTVT [2013-2016],Hatchback,Mumbai,46471,Manual,Petrol,2016,First Owner,550000
2,Hyundai,Verna [2011-2015],Fluidic 1.4 VTVT,Sedan,Bhopal,57313,Manual,Petrol,2012,First Owner,450000
3,Hyundai,Elite i20 [2014-2015],Asta 1.2,Hatchback,Varanasi,62000,Manual,Petrol,2015,First Owner,475000
4,Honda,Amaze [2016-2018],1.2 E i-VTEC,-,Kanpur,31000,Manual,Petrol,2019,First Owner,570000
...,...,...,...,...,...,...,...,...,...,...,...
50384,Mercedes-Benz,C-Class [2014-2018],C 220 CDI Style,Sedan,Mumbai,27000,Automatic,Diesel,2018,Second Owner,3075000
50385,Audi,A6[2011-2015],3.0 TDI quattro Premium Plus,Sedan,Noida,76000,Automatic,Diesel,2013,Second Owner,1250000
50386,Mercedes-Benz,E-Class [2009-2013],E350,Sedan,Mumbai,55000,Automatic,Petrol,2010,Second Owner,990000
50387,Maruti Suzuki,Baleno [2015-2019],Delta 1.2 AT,Hatchback,Meerut,32000,Automatic,Petrol,2016,First Owner,550000


In [91]:
#Function to collect car info from Cars24
def get_cars_24_info(driver,url):    
    #Get car info from json response
    try:
        json_data=car_info_json_using_webdriver(driver,url)
    except:
        raise("Expection while collecting json data")
    #Empty list for collecting brand info
    brand_list=[]
    #Empty list for collecting model info
    model_list=[]
    #Empty list for collecting variant info
    variant_list=[]
    #Empty list for collecting body type info
    body_type_list=[]
    #Empty list for collecting location info
    location_list=[]
    #Empty list for collecting km driven info
    km_driven_list=[]
    #Empty list for collecting no.of.owners info
    no_of_owners_list=[]
    #Empty list for collecting transmission type info
    transmission_type_list=[]
    #Empty list for collecting fuel type info
    fuel_type_list=[]
    #Empty list for collecting manufacturing year info
    manf_year_list=[]
    #Empty list for collecting price info
    price_list=[]
    #print(len(json_data['data']['cars']))
    for car in json_data['data']['content']:
        try:
            brand_list.append(car['make'])
        except:
            brand_list.append('-')
        try:
            model_list.append(car['model'])
        except:
            model_list.append('-')
        try:
            variant_list.append(car['variant'])
        except:
            variant_list.append('-')
        try:
            body_type_list.append(car['bodyType'])
        except:
            body_type_list.append('-')
        try:
            transmission_type_list.append(car['transmission'])
        except:
            transmission_type_list.append('-')
        try:
            no_of_owners_list.append(car['ownerNumber'])
        except:
            no_of_owners_list.append('-')
        try:
            location_list.append(car['city'])
        except:
            location_list.append('-')
        try:
            km_driven_list.append(car['kilometerDriven'])
        except:
            km_driven_list.append('-')
        try:
            fuel_type_list.append(car['fuelType'])
        except:
            fuel_type_list.append('-')
        try:
            manf_year_list.append(car['year'])
        except:
            manf_year_list.append('-')
        try:
            price_list.append(car['price'])
        except:
            price_list.append('-')
    car_table=pd.DataFrame({"Brand":brand_list,"Model":model_list,"Variant":variant_list,"Body Type":body_type_list,
                            "Location":location_list,"Km Driven":km_driven_list,"Transmission Type":transmission_type_list,
                            "Fuel Type":fuel_type_list,"Manufacture Year":manf_year_list,"No.Of.Owners":no_of_owners_list,"Price":price_list
                           })
    return car_table

In [92]:
#Function to scrap car info from Cars24
def car_24_cars():
    #Api url for getting Json reponse
    j_url='https://api-sell24.cars24.team/buy-used-car?page=0'
    #Initiating selenium Webdriver
    driver=initiate_driver() 
    #Get total number of pages
    page_count=int(car_info_json_using_webdriver(driver,j_url)['data']['totalPages'])
    print(page_count)
    i=0
    #Creating empty data frame to append the results
    cars_df=pd.DataFrame()
    #looping until we reach max page number
    while(i<=page_count):
        j_url=j_url[:j_url.rindex('page=')]+'page='+str(i)
        print(j_url)
        try:
            #Keep appending the result of each url to the final result dataframe
            cars_df=cars_df.append(get_cars_24_info(driver,j_url),ignore_index=True)
        except:
            print("Exception occured whlile collecting information")
        i=i+1
    print(cars_df.shape)
    cars_df.to_csv('C:/Users/yn/Desktop/Yuvi/DataTrained/InternshipProjects/CarPricePrediction/CarPriceDataSetCars24.csv')
    return cars_df

In [93]:
car_24_cars()

289
https://api-sell24.cars24.team/buy-used-car?page=0
https://api-sell24.cars24.team/buy-used-car?page=1
https://api-sell24.cars24.team/buy-used-car?page=2
https://api-sell24.cars24.team/buy-used-car?page=3
https://api-sell24.cars24.team/buy-used-car?page=4
https://api-sell24.cars24.team/buy-used-car?page=5
https://api-sell24.cars24.team/buy-used-car?page=6
https://api-sell24.cars24.team/buy-used-car?page=7
https://api-sell24.cars24.team/buy-used-car?page=8
https://api-sell24.cars24.team/buy-used-car?page=9
https://api-sell24.cars24.team/buy-used-car?page=10
https://api-sell24.cars24.team/buy-used-car?page=11
https://api-sell24.cars24.team/buy-used-car?page=12
https://api-sell24.cars24.team/buy-used-car?page=13
https://api-sell24.cars24.team/buy-used-car?page=14
https://api-sell24.cars24.team/buy-used-car?page=15
https://api-sell24.cars24.team/buy-used-car?page=16
https://api-sell24.cars24.team/buy-used-car?page=17
https://api-sell24.cars24.team/buy-used-car?page=18
https://api-sell24

,Brand,Model,Variant,Body Type,Location,Km Driven,Transmission Type,Fuel Type,Manufacture Year,No.Of.Owners,Price
0,HYUNDAI,GRAND I10,SPORTZ 1.2 KAPPA VTVT,Hatchback,Ahmedabad,85892,Manual,Petrol,2013,1,377799
1,MARUTI,ALTO 800,LXI,Hatchback,Ahmedabad,53590,Manual,Petrol,2019,1,322399
2,MARUTI,ALTO 800,VXI,Hatchback,Patna,31568,Manual,Petrol,2018,1,319799
3,MARUTI,ALTO K10,VXI,Hatchback,Patna,20359,Manual,Petrol,2018,1,347299
4,HYUNDAI,EON,ERA PLUS,Hatchback,Kochi,47751,Manual,Petrol,2017,2,298899
...,...,...,...,...,...,...,...,...,...,...,...
5780,FORD,FIESTA,1.6 SXI ABS,Sedan,New Delhi,95318,Manual,Petrol,2007,1,100000
5781,MARUTI,RITZ,VDI,Hatchback,Ahmedabad,89043,Manual,Diesel,2011,2,230000
5782,MARUTI,SWIFT,ZXI 1.3,Hatchback,New Delhi,61167,Manual,Petrol,2009,2,210000
5783,HYUNDAI,GRAND I10,MAGNA 1.1 CRDI,Hatchback,Ghaziabad,82685,Manual,Diesel,2013,3,300000


In [108]:
#Collect car info from OLX
def get_olx_info(driver,url):    
    #Get car info from json response
    try:
        json_data=car_info_json_using_webdriver(driver,url)
    except:
        raise("Exception occured while collecting info")
    #Empty list for collecting brand info
    brand_list=[]
    #Empty list for collecting model info
    model_list=[]
    #Empty list for collecting variant info
    variant_list=[]
    #Empty list for collecting bodyType info
    body_type_list=[]
    #Empty list for collecting location info
    location_list=[]
    #Empty list for collecting KmDriven info
    km_driven_list=[]
    #Empty list for collecting No.Of.Owners info
    no_of_owners_list=[]
    #Empty list for collecting Transmission Type info
    transmission_type_list=[]
    #Empty list for collecting Fuel type info
    fuel_type_list=[]
    #Empty list for collecting manufacturing year info
    manf_year_list=[]
    #Empty list for collecting price info
    price_list=[]
    #print(len(json_data['data']['cars']))
    for car in json_data['data']:
        if(len(car['parameters'])==8):
            try:
                brand_list.append(car['parameters'][0]['value_name'])
            except:
                brand_list.append('-')
            try:
                model_list.append(car['parameters'][1]['value_name'])
            except:
                model_list.append('-')
            try:
                variant_list.append(car['parameters'][2]['value_name'])
            except:
                variant_list.append('-')
            try:
                body_type_list.append(car['car_body_type'])
            except:
                body_type_list.append('-')
            try:
                transmission_type_list.append(car['parameters'][5]['value_name'])
            except:
                transmission_type_list.append('-')
            try:
                no_of_owners_list.append(car['parameters'][7]['value_name'])
            except:
                no_of_owners_list.append('-')
            try:
                location_list.append(car['locations_resolved']['ADMIN_LEVEL_3_name'])
            except:
                location_list.append('-')
            try:
                km_driven_list.append(car['parameters'][6]['value_name'])
            except:
                km_driven_list.append('-')
            try:
                fuel_type_list.append(car['parameters'][4]['value_name'])
            except:
                fuel_type_list.append('-')
            try:
                manf_year_list.append(car['parameters'][3]['value_name'])
            except:
                manf_year_list.append('-')
            try:
                price_list.append(car['price']['value']['raw'])
            except:
                price_list.append('-')
        else:
            continue
    car_table=pd.DataFrame({"Brand":brand_list,"Model":model_list,"Variant":variant_list,"Body Type":body_type_list,
                            "Location":location_list,"Km Driven":km_driven_list,"Transmission Type":transmission_type_list,
                            "Fuel Type":fuel_type_list,"Manufacture Year":manf_year_list,"No.Of.Owners":no_of_owners_list,"Price":price_list
                           })
    return car_table

In [110]:
#Function to scrap olx cars
def olx_cars():
    #Api url for getting Json response
    j_url='https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=4059223&page=0'
    #Initiating selenium webdriver
    driver=initiate_driver()
    #Loading html url to collect the location list
    driver.get('https://www.olx.in/cars_c84')
    #Finding list of locations in website
    location_list=driver.find_elements_by_xpath('//a[@data-aut-id="location_India"]/following-sibling::ul[1]//li/a')
    location_dict={}
    #Collect location id from website
    for loc in location_list:
        loc_id=loc.get_attribute('href').split('/cars')[0]
        location_id=loc_id[loc_id.rindex('_g'):].replace('_g','')
        location_name=loc.get_attribute('data-aut-id').replace('location_','')
        location_dict[location_id]=location_name
    #Creating empty Dataframe
    cars_df=pd.DataFrame()   
    print(len(location_dict))
    #looping through location ids as the api provides max of 1000 item per listing
    for key,value in location_dict.items():
        j_url=j_url.split('&location=')[0]+'&location='+key+j_url[j_url.rindex('&page='):]
        print(j_url)
        i=0
        while(i<10):
            j_url=j_url.split('page=')[0]+'page='+str(i)
            #Keep appending the dataframe to the main dataframe to get the final result in single dataframe
            try:
                cars_df=cars_df.append(get_olx_info(driver,j_url),ignore_index=True)
            except:
                print("Exception ocuured while collecting information from OLX.")
            i=i+1
    print(cars_df.shape)
    #Writing the result dataframe to csv file
    cars_df.to_csv('C:/Users/yn/Desktop/Yuvi/DataTrained/InternshipProjects/CarPricePrediction/CarPriceDataSetOlx.csv')
    return cars_df

In [111]:
olx_cars()

20
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001163&page=0
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001173&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001160&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001176&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001152&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001159&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001154&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001170&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2007599&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001155&page=9
https://www.olx.in/api/relevance/v2/search?category=84&size=100&location=2001145&page=9
https://www.olx.in/api/releva

,Brand,Model,Variant,Body Type,Location,Km Driven,Transmission Type,Fuel Type,Manufacture Year,No.Of.Owners,Price
0,Hyundai,i20,2010-2012 1.2 Magna,HATCHBACK,Mumbai,37500,Manual,Petrol,2011,2nd,244555.0
1,Toyota,Innova,2004-2011 2.5 G4 Diesel 8-seater,SUV,Mumbai,137563,Manual,Diesel,2009,1st,555500.0
2,Maruti Suzuki,Swift,VXI,HATCHBACK,Mumbai,65900,Manual,Petrol,2009,1st,215000.0
3,Toyota,Camry,2002-2011 A/T,HYBRID_SEDAN,Pune,100000,Automatic,Petrol,2007,3rd,375000.0
4,Toyota,Innova,2.5 V 7 STR,SUV,Mumbai,83000,Manual,Diesel,2014,1st,954000.0
...,...,...,...,...,...,...,...,...,...,...,...
18026,Maruti Suzuki,Alto K10,LX,HATCHBACK,Dehradun,60000,Manual,Petrol,2007,3rd,130000.0
18027,Hyundai,i10,Magna,HATCHBACK,Dehradun,60000,Manual,Petrol,2011,1st,225000.0
18028,Honda,City ZX,VTEC,SEDAN,Dehradun,11000,Manual,Petrol,2007,3rd,135000.0
18029,Mahindra,Jeep,2.5L,SUV,Vikasnagar,80000,Manual,Diesel,1979,2nd,260000.0
